In [1]:
from xml.etree import ElementTree as ET

def extract_text(file_path, output_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    sentences = []
    for seg in root.iter('seg'):
        sentences.append(seg.text)
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(sentences))

extract_text("fr-en/IWSLT13.TED.dev2010.fr-en.fr.xml", "fr-en/dev2010.fr")
extract_text("fr-en/IWSLT13.TED.dev2010.fr-en.en.xml", "fr-en/dev2010.en")
extract_text("fr-en/IWSLT13.TED.tst2010.fr-en.fr.xml", "fr-en/test2010.fr")
extract_text("fr-en/IWSLT13.TED.tst2010.fr-en.en.xml", "fr-en/test2010.en")


In [3]:
!pip show sentencepiece


Name: sentencepiece
Version: 0.2.0
Summary: SentencePiece python wrapper
Home-page: https://github.com/google/sentencepiece
Author: Taku Kudo
Author-email: taku@google.com
License: Apache
Location: c:\users\user\anaconda3\envs\pytorch\lib\site-packages
Requires: 
Required-by: hanlp


Error processing line 1 of C:\Users\USER\anaconda3\envs\pytorch\lib\site-packages\zope.interface-7.1.1-py3.8-nspkg.pth:

  Traceback (most recent call last):
    File "C:\Users\USER\anaconda3\envs\pytorch\lib\site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 553, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored


In [7]:
import sentencepiece as spm
print("SentencePiece imported successfully")


SentencePiece imported successfully


In [9]:
!pip uninstall -y hydra-core omegaconf
!pip install hydra-core==1.1.0 omegaconf==2.1.0


Found existing installation: hydra-core 1.0.7
Uninstalling hydra-core-1.0.7:
  Successfully uninstalled hydra-core-1.0.7
Found existing installation: omegaconf 2.3.0
Uninstalling omegaconf-2.3.0:
  Successfully uninstalled omegaconf-2.3.0
  Using cached antlr4_python3_runtime-4.8-py3-none-any.whl
   ---------------------------------------- 0.0/144.6 kB ? eta -:--:--
   ----- --------------------------------- 20.5/144.6 kB 640.0 kB/s eta 0:00:01
   ---------------------------------- ----- 122.9/144.6 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 144.6/144.6 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/74.5 kB ? eta -:--:--
   ---------------------------------------- 74.5/74.5 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: antlr4-python3-runtime
    Found existing installation: antlr4-python3-runtime 4.9.3
    Uninstalling antlr4-python3-runtime-4.9.3:
      Successfully uninstalled antlr4-python3-runtime-4.9.3


DEPRECATION: omegaconf 2.1.0 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fairseq 0.12.2 requires hydra-core<1.1,>=1.0.7, but you have hydra-core 1.1.0 which is incompatible.
fairseq 0.12.2 requires omegaconf<2.1, but you have omegaconf 2.1.0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
!pip show hydra-core
!pip show omegaconf


Name: hydra-core
Version: 1.1.0
Summary: A framework for elegantly configuring complex applications
Home-page: https://github.com/facebookresearch/hydra
Author: Omry Yadan
Author-email: omry@fb.com
License: MIT
Location: c:\users\user\anaconda3\envs\pytorch\lib\site-packages
Requires: antlr4-python3-runtime, importlib-resources, omegaconf
Required-by: fairseq
Name: omegaconf
Version: 2.1.0
Summary: A flexible configuration library
Home-page: https://github.com/omry/omegaconf
Author: Omry Yadan
Author-email: omry@yadan.net
License: UNKNOWN
Location: c:\users\user\anaconda3\envs\pytorch\lib\site-packages
Requires: antlr4-python3-runtime, PyYAML
Required-by: fairseq, hydra-core


In [15]:
import sentencepiece as spm

# 訓練 BPE 模型
spm.SentencePieceTrainer.train(
    input="fr-en/train.fr,fr-en/train.en",
    model_prefix="fr-en/spm",
    vocab_size=32000,
    character_coverage=1.0,
    model_type="bpe"
)

# 使用模型進行編碼
def encode_file(input_file, output_file, spm_model):
    sp = spm.SentencePieceProcessor(model_file=spm_model)
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
        for line in infile:
            outfile.write(' '.join(sp.encode(line.strip(), out_type=str)) + '\n')

# 編碼訓練和測試數據
encode_file("fr-en/train.fr", "fr-en/train.bpe.fr", "fr-en/spm.model")
encode_file("fr-en/train.en", "fr-en/train.bpe.en", "fr-en/spm.model")
encode_file("fr-en/test2010.fr", "fr-en/test2010.bpe.fr", "fr-en/spm.model")
encode_file("fr-en/test2010.en", "fr-en/test2010.bpe.en", "fr-en/spm.model")


In [19]:
def align_files(file1, file2, output1, output2):
    with open(file1, 'r', encoding='utf-8') as f1, open(file2, 'r', encoding='utf-8') as f2:
        lines1 = f1.readlines()
        lines2 = f2.readlines()
    
    # 對齊行數
    min_len = min(len(lines1), len(lines2))
    lines1 = lines1[:min_len]
    lines2 = lines2[:min_len]
    
    with open(output1, 'w', encoding='utf-8') as out1, open(output2, 'w', encoding='utf-8') as out2:
        out1.writelines(lines1)
        out2.writelines(lines2)

# 修復 train.bpe 文件
align_files("fr-en/train.bpe.fr", "fr-en/train.bpe.en", "fr-en/train.bpe.aligned.fr", "fr-en/train.bpe.aligned.en")


In [20]:
import subprocess

# 預處理命令
preprocess_command = [
    "fairseq-preprocess",
    "--source-lang", "fr",
    "--target-lang", "en",
    "--trainpref", "fr-en/train.bpe.aligned",
    "--validpref", "fr-en/dev2010.bpe",
    "--testpref", "fr-en/test2010.bpe",
    "--destdir", "data-bin/fr-en",
    "--workers", "4",
    "--joined-dictionary"
]

# 執行預處理
result = subprocess.run(preprocess_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# 打印輸出
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)


STDOUT: 
STDERR: 2025-01-16 22:24:31 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\envs\pytorch\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\USER\anaconda3\envs\pytorch\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\USER\anaconda3\envs\pytorch\Scripts\fairseq-preprocess.exe\__main__.py", line 4, in <module>
    from fairseq_cli.preprocess import cli_main
  File "C:\Users\USER\anaconda3\envs\pytorch\lib\site-packages\fairseq_cli\preprocess.py", line 18, in <module>
    from fairseq import options, tasks, utils
  File "C:\Users\USER\anaconda3\envs\pytorch\lib\site-packages\fairseq\__init__.py", line 40, in <module>
    import fairseq.scoring  # noqa
  File "C:\Users\USER\anaconda3\envs\pytorch\lib\site-packages\fairseq\scoring\__init__.py", line 55, in <module>
    importlib

In [23]:
import lxml.etree
print("lxml is installed and ready to use!")


ModuleNotFoundError: No module named 'lxml.etree'

In [25]:
!pip install --upgrade pip setuptools wheel


  Using cached pip-24.3.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-24.3.1-py3-none-any.whl (1.8 MB)


ERROR: To modify pip, please run the following command:
C:\Users\USER\anaconda3\envs\pytorch\python.exe -m pip install --upgrade pip setuptools wheel

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
!pip install lxml


  Using cached lxml-5.3.0-cp38-cp38-win_amd64.whl.metadata (3.9 kB)
Using cached lxml-5.3.0-cp38-cp38-win_amd64.whl (3.8 MB)


DEPRECATION: omegaconf 2.1.0 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
!pip uninstall -y lxml
!pip install lxml --no-binary :all:


Found existing installation: lxml 5.3.0
Uninstalling lxml-5.3.0:
  Successfully uninstalled lxml-5.3.0
     ---------------------------------------- 0.0/3.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.7 MB 435.7 kB/s eta 0:00:09
     ---- ----------------------------------- 0.4/3.7 MB 3.9 MB/s eta 0:00:01
     ------------- -------------------------- 1.3/3.7 MB 8.9 MB/s eta 0:00:01
     ------------------------- -------------- 2.3/3.7 MB 12.3 MB/s eta 0:00:01
     -------------------------------------- - 3.5/3.7 MB 14.9 MB/s eta 0:00:01
     ---------------------------------------- 3.7/3.7 MB 14.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  pip subprocess to install build dependencies did not run successfully.
  exit code: 1
  
  [403 lines of output]
       ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
       - -------------------------------------- 0.1/2.8 MB 1.3 MB/s eta 0:00:03
       ----------------------- ---------------- 1.6/2.8 MB 17.4 MB/s eta 0:00:01
       ---------------------------------------- 2.8/2.8 MB 22.0 MB/s eta 0:00:00
    Preparing metadata (setup.py): started
    Preparing metadata (setup.py): finished with status 'done'
       ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
       ---------------------------------------- 1.4/1.4 MB 41.9 MB/s eta 0:00:00
    Getting requirements to build wheel: started
    Getting requirements to build wheel: finished with status 'done'
    Preparing metadata (pyproject.toml): started
    Preparing metadata (pyproject.toml): finished with status 'done'
       --------------------------

In [ ]:
!pip list
